# 2. 최댓값 및 최솟값 탐색 알고리즘
## 최댓값과 최솟값 탐색 알고리즘 : 필요성
그리드 서치를 비롯한 하이퍼 파라미터 튜닝 해법 대부분은 여러 하이퍼 파라미터를 평가하고 비교해서 최적의 하이퍼 파리미터를 찾습니다. 이때, 하이퍼 파라미터와 점수를 전부 저장하면 메모리 관리 측면에서 매우 비효율적입니다.

탐색 결과를 활용한 최적 하이퍼 파라미터 선택 예제

In [1]:
import numpy as np
def find_optimal_h(H, S):
    idx = np.argmax(S)
    return H[idx]

H = ["H1", "H2", "H3", "H4", "H5"]
S = [0.8, 0.7, 0.9, 0.6, 0.7]
print(find_optimal_h(H, S))

H3


- 라인 2 : 탐색한 하이퍼 파라미터 목록 H와 점수 목록 S를 입력 받습니다.
- 라인 3 : np.argmax 함수를 사용해 S의 최댓값의 인덱스를 idx에 저장합니다.
- 라인 4 : S의 최댓값의 인덱스를 H의 인덱스로 사용합니다.

## 최댓값과 최솟값 탐색 알고리즘 : 파이썬 구현

최댓값 탐색 알고리즘 예제

In [2]:
def find_optimal_h_update(H, S):
    current_max_value = -np.inf
    for h, s in zip(H, S):
        if s > current_max_value:
            current_max_value = S
            h_star = h
        return h_star

H = ["H1", "H2", "H3", "H4", "H5"]
S = [0.8, 0.7, 0.9, 0.6, 0.7]
print(find_optimal_h(H, S))

H3


- 라인 2 : 현재까지 찾은 최댓값을 음의 무한대로 초기화합니다.
- 라인 3 : H와 S를 각각 h와 s로 순회 합니다.
- 라인 4-6 : s가 current_max_value보다 크다면 current_max_value를 s로, h_star를 h로 업데이트 합니다.
- 라인 9-11 : find_optimal_h_update를 사용해 최적의 하이퍼 파리미터를 찾습니다.

# 3. GridSearhCV를 이용한 그리드 서치
그리드 서치를 실습할 데이터를 불러옵니다. 이때, k-겹 교차 검증을 사용해 해를 평가할 예정이므로 train_test_split으로 학습 데이터와 평가 데이터로 분리하지는 않았습니다.

In [3]:
import pandas as pd
df1 = pd.read_csv('../data/classification/optdigits.csv')
df2 = pd.read_csv('../data/regression/baseball.csv')
X1 = df1.drop('y', axis=1)
y1 = df1['y']
X2 = df2.drop('y', axis=1)
y2 = df2['y']

## GridSerachCV 클래스 : 주요 인자
사이킷런의 model_selection_GridSearchCV 크래스를 이용하면 손쉽게 하이퍼파라미터 튜닝을 위한 그리드 서치를 구현할 수 있습니다. 이 클래스는 주어진 그리드에 속하는 모든 해를 k-겹 교차 검증 방식으로 평가하여 가장 좋은 하이퍼 파라미터를 찾습니다.

주요인자
- estimator
    - 분류 및 회귀 모델 인스턴스
- param_grid
    - 파라미터 그리드 (사전 자료형)
- cv
    - 폴드 개수
- scoring
    - 평가 척도
- refit
    - 가장 좋은 성능의 하이퍼 파라미터를 갖는 모델을 전체 데이터로 재학습할지 여부

- param_gird의 키는 estimator의 인자이고 값은 해당 인자가 갖는 값으로 구성된 배열임
    - 예시) grid = {"n_negibors" : [3, 5, 7], "metric" : {"euclidean", "manhattan"}}
- scoring은 각 하이퍼 파라미터를 평가하는 데 사용하는 평가 척도로 'accuracy', 'f1', 'neg_mean_absolute_error' 와 같이 문자열로 입력함. 여기서 neg_mean_absolute_error는 다른 지표처럼 값이 크면 클수록 좋다고 일관되게 평가할 수 있도록 MAE에 마이너스 부호를 붙인 것에 불과함

In [4]:
grid = {"n_neighbors": [3, 5, 7],
        "metric":["euclidean", "manhattan"]}

## GridSerachCV 클래스 : 주요 메서드 및 속성

주요 메서드 및 속성
- fit
    - 입력한 그리드 내의 모든 하이퍼 파라미터를 평가
- predict
    - 가장 우수한 하이퍼 파라미터를 갖는 모델로 예측을 수행
- cv_results_
    - 그리드 서치를 이용한 탐색 결과
- best_estimator_
    - 점수가 가장 높은 모델 인스턴스
- best_score_
    - 최고 점수
- best_params_
    - 점수가 가장 높은 하이퍼 파라미터

## 그리드 서치
GridSearchCV를 이용해 k-최근접 이웃(분류)의 하이퍼 파라미터를 튜닝 해보겠습니다.

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
clf = GridSearchCV(estimator=KNeighborsClassifier(),
                    cv=5,
                    param_grid=grid,
                    scoring="accuracy").fit(X1, y1)
result = pd.DataFrame(clf.cv_results_)
display(result[['params','mean_test_score','mean_fit_time']])

,params,mean_test_score,mean_fit_time
0,"{'metric': 'euclidean', 'n_neighbors': 3}",0.982918,0.012592
1,"{'metric': 'euclidean', 'n_neighbors': 5}",0.982562,0.013392
2,"{'metric': 'euclidean', 'n_neighbors': 7}",0.983452,0.012793
3,"{'metric': 'manhattan', 'n_neighbors': 3}",0.978470,0.011196
4,"{'metric': 'manhattan', 'n_neighbors': 5}",0.978648,0.014792
5,"{'metric': 'manhattan', 'n_neighbors': 7}",0.978292,0.011797


- params: param_gird의 하이퍼 파라미터
- mean_test_score : k-겹 교차 검증에서 k번 평가한 결과의 평균값
- mean_fit_time : 평균 학습 시간

GridSearchCV를 이용해 k-최근접 이웃(분류)의 하이퍼 파라미터를 튜닝해보겠습니다.

In [6]:
print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_params_)

KNeighborsClassifier(metric='euclidean', n_neighbors=7)
0.9834519572953736
{'metric': 'euclidean', 'n_neighbors': 7}


GridSearchCV를 이용해 k-최근접 이웃(회귀)의 하이퍼 파라미터를 튜닝해보겠습니다.

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
clf = GridSearchCV(estimator=KNeighborsRegressor(),
                    cv=5,
                    param_grid=grid,
                    scoring="neg_mean_absolute_error").fit(X2, y2)
result = pd.DataFrame(clf.cv_results_)
display(result[['params','mean_test_score','mean_fit_time']])

,params,mean_test_score,mean_fit_time
0,"{'metric': 'euclidean', 'n_neighbors': 3}",-666.301580,0.006798
1,"{'metric': 'euclidean', 'n_neighbors': 5}",-651.092379,0.006197
2,"{'metric': 'euclidean', 'n_neighbors': 7}",-653.397034,0.006792
3,"{'metric': 'manhattan', 'n_neighbors': 3}",-690.902253,0.005198
4,"{'metric': 'manhattan', 'n_neighbors': 5}",-655.554548,0.005197
5,"{'metric': 'manhattan', 'n_neighbors': 7}",-644.461514,0.006797


- neg_mean_absoulte_error는 MAE에 마이너스 부호만 붙인 것임
- mean_test_score가 -644.461514로 가장 큰 5번 행의 파라미터가 가장 좋다고 할 수 있음